# Projecte de Bases de Dades

Aquest notebook conté els queries que hem usat per obtenir les dades i l'anàlisis d'aquestes.

# Imports

In [ ]:
# Uncomment to install dependencies if necessary
#!pip install PyMySQL
#!pip install scikit-learn
#!pip install nltk
#!pip install wordcloud
#!pip install networkx

In [ ]:
# Draw the plots immediately after the current cell
%matplotlib inline

import pandas as pd
import pymysql
import warnings

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.style.use('default')

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Creació de la base de dades

El backup de la base de dades final es pot trobar en `Database/github.sql`. Aquest conté el schema i les dades; per crear la base de dades només cal executar tot el fitxer (amb el botó ⚡ en MySQL Workbench). La base de dades s'anomena `github`.


# Connexió

Per començar a treballar amb la base de dades, cal connectar-se primer:

In [ ]:
db_name = "github"
db_host = "localhost"
db_port = 3306
db_username = "root"
db_password = input("Enter the DB password")

dataBaseConnection = pymysql.connect(host=db_host,
                            port=db_port,
                            user=db_username,
                            password=db_password,
                            db=db_name)

In [ ]:
# Get the cursor to interact with the database
cursor = dataBaseConnection.cursor()

In [ ]:
def execute_select_query(cursor, query):
    """
    Executes a query and returns the results as a pandas dataframe.
    """
    cursor.execute(query)
    output = cursor.fetchall()

    # Fetch column names from cursor's description
    columns = [desc[0] for desc in cursor.description]
    
    # Convert output to pandas DataFrame
    if output:
        df = pd.DataFrame(output, columns=columns)
    return output, df

# Taules
A continuació es mostren exemples de les entitats dins la base de dades.

In [ ]:
# Repositories
query = "SELECT * FROM Repositories WHERE mainLanguage != \"\""
output_repo, df_repo = execute_select_query(cursor, query)
df_repo.tail(5)

In [ ]:
# Repository visits
query = "SELECT * FROM RepositoryVisits;"

output_repo_visists, df_repo_visists = execute_select_query(cursor, query)
df_repo_visists.head(5)

In [ ]:
# Topics/tags of repositories
query = """
SELECT r.name, r.owner, GROUP_CONCAT(t.topic) as topics FROM Repositories r
JOIN RepositoryTopics t
ON t.repo = r.name AND t.owner = r.owner
GROUP BY r.name, r.owner
"""

output_repo_topics, df_repo_topics = execute_select_query(cursor, query)
df_repo_topics.tail(5)

In [ ]:
# Commit messages
query = "SELECT * FROM Commits;"
output_commits, df_commits = execute_select_query(cursor, query)
df_commits.head(5)

# Anàlisi quantitatiu
A partir de les dades també ens interessa realitzar uns anàlisi quantitatiu:
* Distribució dels temes principals (gràfic de barres o de pastís). És a dir, en quina proporció dels nostres repositoris estudiats es tracten.
* Distribució de llenguatges (gràfic de barres o de pastís). Com l'anterior.

In [ ]:
# RepositoryTopics is the N-N relationship table for Repository-Topic;
# ie. the topics of each repository.
output_repo_topics, df_repo_topics = execute_select_query(cursor, "SELECT * FROM RepositoryTopics;")

In [ ]:
n = 20 # We'll check the top 20 topics used
topics = df_repo_topics["topic"].value_counts().to_frame()
topics_count = topics.sum()
n_topics_count = topics[:n].sum()
n_topics_name = df_repo_topics["topic"].value_counts()[:n].index.to_list()
print("Most used topics (present in {:.2f}% of repositories):".format(float((n_topics_count / topics_count).iloc[0]) * 100))
display(topics[:n])
ax = topics[:n].plot.bar()

En el gràfic de barres anterior es mostra els primers 20 temes més freqüents tractats en els nostres repositoris. No obstant això, només estan presents en el 15% d'ells, la qual cosa ens mostra que la nostra mostra està àmpliament distribuïda.

Analitzem a continuació els llenguatges més usats:

In [ ]:
tmp = df_repo.drop(df_repo[df_repo["mainLanguage"] == ""].index) # Ignore repositories where the main language is unknown
languages = tmp["mainLanguage"].value_counts().to_frame()
display(languages)
languages_count = languages.sum()
n_languages_count = languages[:n].sum()
print("Main topics present in {:.2f}% of repositories.".format(float((n_languages_count / languages_count).iloc[0]) * 100))
print(languages.shape)
display(languages)
languages[:n].plot.bar()

Els llenguatges més utilitzats en la nostra mostra de repositoris són llenguatges orientats al desenvolupament web; aquest resultat és coherent amb el fet que topics/tags per a frameworks de desenvolupament web com React són els més usats. Seguits de Python, Go, Java i C++. Aquests llenguatges més utilitzats representen més del 90% dels repositoris.

In [ ]:
# TODO a similar chart for TopicVisits

# Reducció dimensional per a una visió general dels repositoris

Una pregunta que ens interessava era si el llenguatge dels repositoris té correlació amb les seves mètriques (nombre de commits, stars, followers).

Per contestar aquesta pregunta podem realitzar un Principal Component Analysis (PCA) per tenir una idea de l'estructura de tot el conjunt de dades. Després, coloraríem els punts de dades en funció del llenguatge utilitzat, per exemple, per veure si aquests grups tenen característiques similars i es troben propers en el dataframe o no.

* Crear un dataframe que conté totes les dades de RepositoryVisits amb les dades més actualitzades per a tots els repositoris estudiats.
* Realitzar una estandardització de les dades (és a dir, per evitar que algunes variables com els commits siguin molt més importants que altres com els forks).
* Realitzar una PCA en 2 components sobre aquest.
* Representar els resultats agrupant els punts en funció de diferents criteris:
    + mainLanguage
    + topic


## Estructura general de les dades numèriques dels repositoris

In [ ]:
# Fetch the numerical fields of RepositoryVisits that we'll use for the PCA
query = """
SELECT CONCAT(RepositoryVisits.owner, RepositoryVisits.name) AS id, forks, commits, stars, watchers, contributors, openIssues, closedIssues, mainLanguage
FROM RepositoryVisits
JOIN Repositories ON RepositoryVisits.owner = Repositories.owner AND RepositoryVisits.name = Repositories.name
WHERE CAST(date AS Date) = '2024-04-16'
"""

_, df = execute_select_query(cursor, query)
df.head(5)

In [ ]:
# Keep only the most used languages
top_languages = df_repo['mainLanguage'].value_counts()[:10].index.tolist()

df = df[df['mainLanguage'].isin(top_languages)]

# Remove outliers
for column in df:
    if (df[column].dtype) != "O":
        q_low = df[column].quantile(0.05)
        q_high = df[column].quantile(0.95)
        # print(q_low)
        # print(q_high)
        df = df[(df[column] <= q_high) & (df[column] >= q_low)]
df.head(5)

In [ ]:
labels = df["id"]
print(labels[0:5])

In [ ]:
# Split numerical data.
X = df.iloc[:,1:-1].values
print(X[:5,:])

In [ ]:
# Standarize each attribute
X_std = StandardScaler().fit_transform(X)
print(X_std[0:5])

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_std)
print(pca.components_)
print(sum(pca.explained_variance_ratio_))
print(X_pca[:5])
print(max(X_std[:,0]))

In [ ]:
top_languages = df_repo['mainLanguage'].value_counts()[:10].index.tolist()
cmap = plt.get_cmap('viridis')
colors = [cmap(i) for i in np.linspace(0, 1, 10)]
#color_map = dict(zip(l, colors))

# Colors to use for each language in the scatterplot
color_map = {
 'C': (0,0.9,1,1),
 'C#': (0.79,0,1,1),
 'C++': (0,0.4,1,1),
 'Go': (1,0.6,0,1),
 'HTML': (1,0.94,0,1),
 'Java': (1,0,0,1),
 'JavaScript': (1,1,0,1),
 'Python': (0,1,0.2,1),
 'Shell': (0.6,0.6,0.6,1),
 'TypeScript': (0.9,1,0,1),
 'PHP': (0.5,1,0,1),
}

In [ ]:
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c = df["mainLanguage"].map(color_map), alpha=0.5)

legend_handles = [mpatches.Patch(color=color_map[language], label=language)
 for language in top_languages]
plt.legend(handles=legend_handles, title='Languages')
plt.show()

# Repositoris "Open Source" & dades per a la pàgina web
Quant a l'objectiu de destacar repositoris open-source (i no només repositoris populars, com fa GitHub), volem exportar dades d'aquests repositoris en format `.json` per a que puguin ser usades per la pàgina web.

Considerem que repositoris són projectes open-source si tenen més de 5 contribuïdors i més de 50 issues en total - un criteri simple però efectiu.

Recollir les dades necessàries involucra fer JOINs amb múltiples taules.

In [ ]:
open_source_projects_query = """
SELECT r.owner, r.name, MAX(r.description) as description, r.mainLanguage,
MAX(stars) as total_stars, MAX(contributors) as total_contributors, MAX(openIssues + closedIssues) as total_issues,
MAX(o.avatar_url) as avatar_url, GROUP_CONCAT(DISTINCT t.topic) as topics,
MAX(watchers) as total_watchers FROM Repositories r
-- Join with RepositoryVisits to get metrics like contributors, stars, issues amount
JOIN RepositoryVisits v
ON r.owner = v.owner AND r.name = v.name
-- Join with Owners to get the avatar URL
JOIN Owners o
ON r.owner = o.username
-- Join with RepositoryTopics to get a list of the tags/topics of the repos
JOIN RepositoryTopics t
ON t.repo = r.name AND t.owner = r.owner
GROUP BY r.owner, r.name, r.mainLanguage, r.license -- Must group by mainLanguage and license as well as they're non-aggregated
HAVING total_contributors > 5 AND total_issues > 50
ORDER BY total_contributors DESC; -- Show repos with most contributors first
"""


output_repo, df_repo = execute_select_query(cursor, open_source_projects_query)
print(len(df_repo), "repositories matching criteria")
df_repo.head(100)

Les dades d'aquests repositoris s'exporten a .json perque puguin ser usades per la pàgina web.

En aquest procés també categoritzem els repositoris segons les seves temàtiques generals:
- Desenvolupament web
- Data science
- Aplicacions
- Eines de desenvolupament
- Repositoris de recursos (ex. una col·lecció d'algorismes)

La categorització es fa segons els "topics" (tags) que tenen els repositoris. Els repositoris també es categoritzen pel seu llenguatge de programació principal.

In [ ]:
from collections import defaultdict
import json

used_languages = set()
used_tags = set()

# Renames or discards languages.
# Used to group up frameworks, variants and transpilers.
LANGUAGE_REMAP = {
    "TypeScript": "JavaScript",
    "Vue": "JavaScript",
    "HTML": "JavaScript", # Bruh
    "Jupyter Notebook": "Python",
    "Kotlin": "Java & Kotlin",
    "Java": "Java & Kotlin",
    "C": "C/C++",
    "C++": "C/C++",
    "Ruby": "Others",
    "Go": "Others",
    "Swift": "Others",
    "Clojure": "Others",
    "Haskell": "Others",
    "Dart": "Others",
    "Shell": "Others",
    "PowerShell": "Others",
    "Scala": "Others",
    "Svelte": "Others",
    "Vim Script": "",
    "CSS": "",
    "SCSS": "",
    "MDX": "",
}

# Remaps GitHub topics to the tags that are used in the website.
TAG_MAP = {}
# Maps tags used by the website to a list of Github "topics" that will be considered as the same tag.
# Ex. repos with topics "react", "vue" become tagged as "Web"
TAG_ALIASES = {
    "Web": ["react", "vue", "web", "reactjs", "css", "chrome-extension", "react-grid", "react-table", "php", "http", "nodejs", "typescript", "electron", "search-engine", "webgl", "rest", "rest-api", "swagger", "static-site-generator", "blog-engine", "router", "webview", "jquery", "http-client", "website", "reactive-templates", "nuxt", "nat", "javascript", "http2", "nginx", "apache", "aws", "api-gateway", "jekyll", "bootstrap"],
    "Modding": ["mod", "minecraft", "emulation", "emulator", "forge", "minecraft-launcher", "modrinth", "minecraft-api", "minecraft-server", "bepinex", "unity3d", "unreal", "unity-mono", "craftbukkit", "valheim", "minecraft-mod", "gta5", "fabric", "gamedev", "retroarch", "game"],
    "Data Science": ["math", "numpy", "data-science", "graphql", "data-visualization", "jupyter-notebook", "big-data"],
    "Machine Learning": ["ml", "pytorch", "deep-learning", "machine-learning", "deep-neural-networks", "tensorflow", "neural-network", "tensor", "computer-vision", "reinforcement-learning", "hyperparameter-tuning", "ai", "artificial-intelligence", "llama", "llms", "llm", "openai"],
    "Tool": ["containers", "zsh", "docker", "github", "cli", "searchengine", "postgrest", "devtool", "cloudstorage", "git", "npm", "database", "postgresql", "backend", "shell-scripting", "websocket", "collaboration", "developer-tools", "promise", "api", "testing", "translation", "i18n", "language", "golang-library", "algorithm", "firmware", "style-linter", "linting", "converter", "blockchain", "wordpress", "static-site-generator", "blog-engine", "material", "material-design", "framework", "argument-parser", "command-line-parser", "readme-generator", "ssh", "backup", "reverse-engineering", "animation", "sdk", "devops", "jenkins", "documentation", "terminal", "encryption", "scrapers", "3d-printing", "reactive-templates", "image-optimization", "file-server", "nat", "proxy", "shell", "linters", "git-client", "raspberry-pi", "blogging", "npm-cli", "aws", "api-gateway", "decompiler", "kubernetes", "tools"],
    "App": ["note-taking", "productivity", "prest", "download", "latex", "text-editor", "curl", "ftp", "bot", "synchronization", "sqlite", "mattermost", "messaging", "conferencing", "remote-desktop", "emacs", "color-picker", "cli-app", "subtitle-downloader", "decompiler", "mobile-app"],

    "Resource": ["learn-to-code", "freecodecamp", "curriculum", "certification", "learnopengl", "lists", "resources", "resource", "dataset", "public-api", "public-apis", "practice", "interview", "styleguide", "list", "interview-questions", "awesome-list", "principles", "design-patterns"],
}
# Tags manually added to some repositories (which otherwise lack descriptive ones)
MANUAL_TAGS = {
    "minio/minio": ["Machine Learning"],
    "Aliucord/Aliucord": ["Modding"],
    "cli-guidelines/cli-guidelines": ["Resource"],
    "yjs/yjs": ["Tool"],
    "TigerVNC/tigervnc": ["Tool"],
    "ollama/ollama": ["App"],
    "micropython/micropython": ["Tool"], # Python implementation
    "raspberrypi/linux": ["App"],
    "rust-lang/rust": ["Tool"],
    "home-assistant/core": ["Tool"],
    "vuejs/vue-cli": ["Tool", "Web"],
    "remix-run/remix": ["Tool", "Web"],
}
# Same thing as above, but mapping tag to list of repos with it instead, as I realized at the end this would've been more convenient.
MANUAL_TAGS2 = {
    "Tool": ["pytorch/tutorials", "vuejs/core", "google/googletest", "google/guava", "ReactiveX/RxJava"],
    "Web": ["vuejs/core"],
    "App": ["square/retrofit"],
}
for tag,aliases in TAG_ALIASES.items():
    for alias in aliases:
        TAG_MAP[alias] = tag
for tag,repos in MANUAL_TAGS2.items():
    for repo in repos:
        if repo in MANUAL_TAGS:
            MANUAL_TAGS[repo].append(tag)
        else:
            MANUAL_TAGS[repo] = [tag]

IGNORED_TAGS = defaultdict(int)

oss_repos = {}
for index, row in df_repo.iterrows():
    key = row["owner"] + "/" + row["name"]
    main_language = row["mainLanguage"]
    if main_language in LANGUAGE_REMAP:
        main_language = LANGUAGE_REMAP[main_language]
    repo = {
        "topics": set(),
        "languages": set([main_language] if main_language != "" else []),
        "stars": row["total_stars"],
        "contributors": row["total_contributors"],
        "icon": row["avatar_url"],
        "description": row["description"],
    }
    for topic in str.split(row["topics"], ","):
        if topic in TAG_MAP:
            repo["topics"].add(TAG_MAP[topic])
        else:
            IGNORED_TAGS[tag] += 1
        if key in MANUAL_TAGS:
            for tag in MANUAL_TAGS[key]:
                repo["topics"].add(tag)

    oss_repos[key] = repo
    used_tags = used_tags.union(repo["topics"])
    used_languages.add(main_language)

# Exclude mirrors and other projects that are not contributable projects or unsuitable
BLACKLISTED_REPOS = [
    "gitlabhq/gitlabhq", # Read-only mirror.
    "qemu/qemu", # Read-only mirror.
    "xasset/xasset",# Not english.
    "jynew/jynew", # Unity RPG game framework, documentation in chinese-only though.
    "doocs/advanced-java", # Chinese-only Java interview questions.
    "CyC2018/CS-Notes", # Chinese computer science course resources.
    "apache/kafka", # Read-only mirror.
]
for blacklisted_repo in BLACKLISTED_REPOS:
    del oss_repos[blacklisted_repo]

# Convert sets to lists for json serialization, and add other
# keys the site expects
for key,repo in oss_repos.items():
    repo["owner"] = key.split("/")[0]
    repo["repo"] = key.split("/")[1]
    repo["topics"] = list(repo["topics"])
    repo["languages"] = list(repo["languages"])

# Export the .json
with open("repositories_output.json", "w") as f:
    json.dump(oss_repos, f, indent=2)

print("Valid repositories:", len(oss_repos))
print("Languages used:", used_languages)
print("Tags used:", used_tags)

Com els repositoris en la base de dades foren principalment trobades per scraping de les pàgines "trending" de GitHub, podem concloure que un 50% dels repositoris que GitHub destaca són només repositoris populars de projectes individuals o d'equips petits i no projectes contribuïbles; només uns 1000 repositoris dels 2000 en la base de dades compleixen els requisits que hem imposat.

# Analisis de missatges de commits

En aquesta secció, farem una anàlisi dels missatges que fan els usuaris a l’hora de fer commits en github.
Veurem quines paraules claus usen més i quin és l’estil més usat amb l’objectiu de determinar una bona
forma d’escriure un missatge de commit.

In [ ]:
# Import necessary libraries

from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import warnings
import nltk
from nltk.corpus import wordnet
import unicodedata
import re

"""
If 
!pip install wordcloud 
doesn't work:
from os import path
import sys
print(sys.executable) # use the path

path -m pip install wordcloud
"""

# Suppress warnings
warnings.filterwarnings("ignore")

# Define constants and configurations
STANDARDIZE_SUBSTITUTION = re.compile(r"[!\"$#%&()*+,\-./:;<=>?[\]^_`{|}~]")
CONSECUTIVE_SPACE_SUBSTITUTION = re.compile(r"  +")
LEMMATIZATION_BLACKLIST = {"was", "as"}
WORD_REPLACEMENTS = {
    "read-me": "readme",
    "read.me": "readme",
    "readme.md": "readme"
}
stopwords = set(STOPWORDS)

# stopwords.update(["a"]) # Manually add stopwords if needed

# Ensure NLTK resources are downloaded
# nltk.download()

### Funcions auxiliars

In [ ]:
def remove_accents(input_str):
    """
    Convert accented characters to base characters.
    Source: https://stackoverflow.com/a/1207479
    """
    nfkd_form = unicodedata.normalize('NFKD', input_str).encode("ascii", "ignore")
    return bytes.decode(nfkd_form)

lemmatizer = nltk.wordnet.WordNetLemmatizer()

# Function to get the wordnet POS tag, where POS means Part of Speech, which is basically the grammatical category of a word
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def standardize(word):
    """Standardize a word."""
    word = word.lower()
    word = re.sub(STANDARDIZE_SUBSTITUTION, " ", word)
    word = re.sub(CONSECUTIVE_SPACE_SUBSTITUTION, " ", word)
    word = remove_accents(word)
    word = word.replace("'s", "").replace("#", "").strip()
    words = [WORD_REPLACEMENTS.get(w, w) for w in word.split()]
    
    # Enhanced lemmatization with POS tagging
    words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) if w not in LEMMATIZATION_BLACKLIST else w for w in words]
    
    return " ".join(words)

In [ ]:
print(standardize("thIs a TesT"))
print(standardize("readme.md"))
_standardize_test_words = [
    "Testing @here as#Dasd 333232 tests testings gItHUbs testing added",  # tests -> test due lemmatization
    "Test_ing !!, (asd,as d).",  # Underscore are considered separated words
    "Test łłłł ñaññañ áaaa",  # Removal of accents and non standard characters
    "テスト #asdasd arabian, wolves",
]

for word in _standardize_test_words:
    print(standardize(word))

## Neteja de dades

Com que l'objectiu es analitzar com escriuen les persones, hi ha un conjunt de missatges que no ens interessen. Per això cal excloure tots aquells missatges que no compleixin uns requisits.

Els nostres criteris d'exclusió són:
- Missatges de bots, els quals podem identificar per un autor que tingui en el nom `[bot]` i missatges que tinguin noms com pot ser: `dependabot` o `renovatebot`
- Eliminar files repetides, producte d'error en la recolecció de dades.
- Els missatges automatics que genera github com poden ser: `Merge pull request`, `Merge branch`, `Squash`, `Initial commit` i `Revert`, ja que no són escrits per les persones.

A partir d'aquests criteris, hem realitzat una neteja de les dades.

In [ ]:
# Find all the commits authored by a bot.
from IPython.display import display
pd.set_option('display.max_rows', None)

cond_a = df_commits[df_commits["author"].str.contains("\[bot]|-bot")]
cond_c = df_commits[df_commits["message"].str.contains("dependabot")]
cond_d = df_commits[df_commits["message"].str.contains("renovatebot")]

temp = pd.concat([cond_a])

unique_auth = pd.unique(temp.author)
unique_msg = pd.unique(cond_c.message)

dict = {'bots' : unique_auth}
df_bots = pd.DataFrame(dict)

# displaying the bots names
display(df_bots)

dict = {'bots_message' : unique_msg}
df_bots_msg = pd.DataFrame(dict)

display(df_bots_msg)

In [ ]:
# Filter out the bots
print("With bots", len(df_commits))
df_commits = df_commits[~df_commits["author"].isin(df_bots["bots"])]
print("Without bots", len(df_commits))

# Filter out repeated rows
df_commits.drop_duplicates(inplace=True)

# Filter out the automatic messages
df_commits = df_commits[
    ~(df_commits.message.str.startswith("Merge pull request") |
      df_commits.message.str.startswith("Merge branch") |
      df_commits.message.str.startswith("Merge remote") |
      df_commits.message.str.startswith("Merge commit") |
      df_commits.message.str.startswith("Merge tag") |
      df_commits.message.str.startswith("Squash") |
      df_commits.message.str.startswith("Initial commit") |
      df_commits.message.str.startswith("Revert ") |
      df_commits.message.str.startswith("Add files via upload") |
      df_commits.message.str.startswith("🔄") |
      df_commits.message.str.startswith("[auto]") |
      df_commits.message.str.startswith("[maven-release-plugin]")
     )
]
print("Without automatic messages", len(df_commits))

Aqui podem veure que de 94.872 missatges, hem reduit els missatges usables a 77.729 . Tot i que segur que queden alguns altres missatges automàtics que no hem pogut detectar a temps, però hem cobert la majoria.

## Analisis paraules més usades

Per fer l'anàlisis de les paraules més usades, ho farem de dues formes. 
- Usarem wordcloud, el qual és una representacó gràfica d'un dibuix creat per paraules on segons les ocurrències d'aquestes, són més grans o si apareixen poc, més petites. Obtenint així, una visió ràpida, de quines paraules són les més usades.
- Comptarem les paraules i mostrarem un gràfic dels top n paraules més usats en ordre descendent, segons paraula més usada en missatges individuals o ocurrències totals.

Adicionalment, tenim `obtain_mask()` el qual usant `transform_format()` converteix una imatge blanc i negra en una màscara que podem usar en el wordcloud perquè només apareguin lletres dins de la regió negra.

In [ ]:
def plot_word_cloud(word_cloud, text, save_image=False, image_name="none"):
    """Create and display a word cloud image."""
    word_cloud_output = word_cloud.generate(text)
    plt.imshow(word_cloud_output, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    if save_image:
        word_cloud_output.to_file(f"./Images/wordclouds/{image_name}_word_cloud.png")

def transform_format(val):
    """Transform format for mask creation."""
    return 255 if val == 0 else val

def obtain_mask(mask):
    """Obtain mask for word cloud."""
    trans_mask = np.ndarray((mask.shape[0], mask.shape[1]), np.int32)
    for i in range(len(mask)):
        trans_mask[i] = list(map(transform_format, mask[i]))
    return trans_mask

In [ ]:
# Load all text
all_text = " ".join(commit for commit in df_commits.message)
# Set the wordcloud params
word_cloud = WordCloud(stopwords=stopwords, max_font_size=30, max_words=200, background_color="black")
# Plot the wordcloud
plot_word_cloud(word_cloud, all_text)

Source of the GitHub image: [Icon by Dryicons](https://dryicons.com/icon/square-github-icon-8312)


In [ ]:
# Use a mask and a color map
github_image = np.array(Image.open("./Images/wordclouds/github_square.png"))

word_cloud_mask = obtain_mask(github_image)
word_cloud = WordCloud(stopwords=stopwords, colormap='rainbow', mask=word_cloud_mask, max_font_size=30, max_words=10000, background_color="black")
plot_word_cloud(word_cloud, all_text, True, "git_mask")

In [ ]:
# Use a mask        
git_image = np.array(Image.open("./Images/wordclouds/git.png"))

word_cloud_mask = obtain_mask(git_image)
word_cloud = WordCloud(stopwords=stopwords, mask=word_cloud_mask,colormap='hot', max_font_size=20, max_words=10000, background_color="#474747")
plot_word_cloud(word_cloud, all_text)

Podem veure que les paraules més usades, són: `fix, add, update` sense comptar els stopwords, ja que podem veure que són les paraules més grans. Tot seguit veurem els valors exactes, de les paraules més usades.

In [ ]:
def count_words(df: pd.DataFrame):
    """
    :param df: DataFrame with the messages and associated information
    :return: Dictionary with the format {word: {n_ocur: value, n_messages: value}, ...}
    """
    # Using defaultdict for convenience; we won't have to add keys explicitly
    dicc = defaultdict(lambda: {"n_ocur": 0, "n_messages": 0})

    total_rows = len(df)
    processed_rows_count = 0
    for row in df.iterrows():
        text = row[1][0]
        text = standardize(text)  # Apply standardization
        
        # Split the text, remove digits and skip if empty
        words = text.split(" ")
        words_without_digits = [word for word in words if not re.search(r'\d', word)]
        unique_words = set(words_without_digits)
        
        if len(unique_words) == 1 and '' in unique_words:
            continue
            
        # Times that a word appears and times that appears in different messages.
        for word in words_without_digits:
            dicc[word]["n_ocur"] += 1
        for word in unique_words:
            dicc[word]["n_messages"] += 1

        processed_rows_count += 1
        # print(f"Word {processed_rows_count}/{total_rows} done; {processed_rows_count / total_rows * 100:.2f}%")
    
    return dicc

frame = {'Messages': df_commits.message}

result = pd.DataFrame(frame)

ocurrencesDict = count_words(result)

# Sort the words by occurrences and occurrences in messages.
def obtain_top_n_words(dictionary, N, filter="n_ocur", desc=True):
    # Get all the words and their frequency
    filtered_words = [(word, freq[filter]) for word, freq in dictionary.items() if word not in stopwords]
    
    # Sort from most to least frequent
    sorted_words = sorted(filtered_words, key=lambda x: x[1], reverse=True)
    
    # Take the N most frequent
    return [(word, freq) for word, freq in sorted_words[:N]]

top_words_list_ocur = obtain_top_n_words(ocurrencesDict, 30, "n_ocur", True)
top_words_list_msg = obtain_top_n_words(ocurrencesDict, 30, "n_messages", True)

print("Ocurrences Top", top_words_list_ocur)
print("\nOcurrences per message Top", top_words_list_msg)

# Function to plot the top words
def plot_top_words(topList, yLabel, ax):
    x = [word for word, freq in topList]
    y = [freq for word, freq in topList]
    
    # Making the bar chart on the data
    ax.bar(x, y)
    
    # Giving title to the plot
    ax.set_title("Top paraules més usades")
    ax.set_xticks(x)
    ax.set_xticklabels(x, rotation=90, ha='right')
    # Giving X and Y labels
    ax.set_xlabel("Paraules")
    ax.set_ylabel(yLabel)
     
    # We do not call plt.show() here, as we want to show both plots together

# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(15, 7))

# Plot the top words by occurrences
plot_top_words(top_words_list_ocur, "Ocurrència de paraules", axs[0])

# Plot the top words by occurrences in different messages
plot_top_words(top_words_list_msg, "Ocurrencies de paraules per missatge", axs[1])

# Adjust layout for better spacing
plt.tight_layout()
plt.savefig("plot")
# Show the plots
plt.show()

# We can see that there is no significant difference, since normally, in a commit message, we do not repeat the same word twice.

De les gràfiques, podem veure que en termes quantitatius, són casi idèntics, tenen la mateixa escala i en general, el mateix ordre de les paraules més usades, encara que la paraula fix, és el que té més diferència indicant, que les persones usen més d'un cop la paraula fix en un missatge. Tot i això denota que no se sol repetir les paraules en un mateix text, ja que totes les altres paraules, tenen un valor similar i la gràfica de l'esquerra, compta les paraules totals, mentre que el de la dreta, compta les paraules per missatge.

Les tres paraules més usades, son `update, add, fix`, els quals eren els mateixos que hem vist en el wordcloud. Hem agrupat les paraules segons un possible significat semàntic.

### Cicle de Desenvolupament:

- **Add:** Adició de noves característiques i creixement del projecte.
- **Fix:** Correció d'errors per assegurar l'estabilitat del projecte.
- **Update:** Millores i actualitzacions del codi existent, assegurant que el projecte es mantingui actualitzat amb les millors pràctiques i tecnologies.

### Paraules Complementàries:

- **Remove:** Reflecteix l'eliminació de codi o funcionalitats obsoletes o innecessàries, la qual cosa és crucial per mantenir el codi net i eficient.
- **Use:** Indica la implementació o reutilització de codi o biblioteques, suggerint un ús eficient dels recursos disponibles.
- **Test:** Subratlla la importància de les proves en el cicle de desenvolupament per assegurar la funcionalitat correcta i evitar regressions.
- **Readme:** Reflecteix la documentació del projecte, important per a la col·laboració i la claredat entre els membres de l'equip i els usuaris.
- **Feat:** Similar a "add", mostra l'enfocament en noves funcionalitats.
- **Chore:** Tasques de manteniment o administratives per a la neteja i organització del projecte.
  
### Paraules Clau en el Manteniment i Millora:

- **Version:** Indica canvis en les versions del projecte, reflectint una evolució contínua i la gestió de versions.
- **File:** Pot referir-se a la gestió de fitxers dins del projecte, com la creació, modificació o eliminació de fitxers.
- **Refactor:** Subratlla l'esforç de reorganització del codi per millorar la seva estructura sense canviar-ne el comportament extern.
- **Improve:** Reflecteix les millores contínues del codi o funcionalitats existents.

### Altres Paraules Rellevants:

- **Build, Link , Error:** Aquestes paraules indiquen activitats específiques relacionades amb la compilació del projecte, la gestió d'enllaços, i la correcció d'errors respectivament.
- **CI:** Fa referència a la integració contínua, important per al desplegament automàtic i la verificació contínua de canvis.
- **Config:** Reflecteix la configuració del projecte, que és crucial per a la seva correcta execució i desplegament



## Grups de paraules més usades

En aquesta secció, usarem **ítems freqüents** per trobar patrons els quals els usuaris escriuen. La idea principal, es trobar grups de paraules els quals quan apareix un, apareix un altre. Això indicaria que les paraules estan relacionades o s'utilitzen conjuntament amb freqüència. Aquest procés ens ajuda a identificar associacions o patrons significatius en el text.

In [ ]:
#https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#apriori-frequent-itemsets-via-the-apriori-algorithm
def generate_data_set(df: pd.DataFrame, rmStopWords = True, useStandard = True):
    """
    :param df: DataFrame with the messages and associated information
    :return dataSet: The message of the commit is the transaction and the words the items.
    """
    data_set = []
    total_rows = len(df)
    processed_rows_count = 0
    for row in df.iterrows():
        text = row[1][0]
        if (useStandard):
            text = standardize(text)  # Apply standardization
       
        # Split the words, but don't hold digits.
        words = [word for word in text.split() if not any(char.isdigit() for char in word)]
        if (rmStopWords):
            unique_words = list(set(words) - stopwords)
        else:
            unique_words = list(set(words))
        data_set.append(unique_words)

        processed_rows_count += 1
        # print(f"Word {processed_rows_count}/{total_rows} done; {processed_rows_count / total_rows * 100:.2f}%")
    return data_set

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import math
#!pip install mlxtend

def get_frequent_itemsets(data, min_supp = 0.01):
    """
    To save memory, you may want to represent your transaction data in the sparse format. 
    This is especially useful if you have lots of products and small transactions.
    """
    te = TransactionEncoder()
    oht_ary = te.fit(data).transform(data, sparse=True)
    sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
    
    # total message * support = number of messages which the bundle appears.
    print("We consider that at least the bundle appears in:", math.ceil(len(data) * min_supp), " messages")
    
    # Calculate it using apriori algorithm
    frequent_itemsets = apriori(sparse_df, min_support=min_supp, use_colnames=True, verbose=1)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    return frequent_itemsets

In [ ]:
# Create the data frame using the commits.
frame = {'Messages': df_commits.message}
result = pd.DataFrame(frame)

In [ ]:
# Since the data set is too big, and our computers can't handle it, we are going to use a randomly sampled fraction of it.
partial_data_set = generate_data_set(result.sample(frac =.50))

In [ ]:
frequent_itemsets = get_frequent_itemsets(partial_data_set, 0.001)
# Sort them from more length and then support
frequent_itemsets_sorted = frequent_itemsets.sort_values(by=['length', 'support'], ascending=[False, False])
print(frequent_itemsets_sorted)

In [ ]:
average_support = math.ceil(frequent_itemsets_sorted['support'].mean() * len(partial_data_set))
print("Average amount of support: ", average_support, " messages")

In [ ]:
data_group = frequent_itemsets[(frequent_itemsets['length'] >= 2)]
data_group['itemsets'] = data_group['itemsets'].apply(lambda x: str(sorted(list(x))))
data_group['support messages'] = data_group['support'].apply(lambda x: math.ceil(len(partial_data_set) * x))
data_group = data_group.sort_values(by=['support messages'], ascending=False)
# Show only those groups, who at least, have the average support.
# With this, we can preserve significant data, while reducing the amount of itemsets
data_group = data_group[(data_group['support messages'] >= average_support)]

print(data_group[['itemsets', 'support messages']])

In [ ]:
# Function to group itemsets by starting word
def group_by_starting_word(df):
    grouped_phrases = {}
    for itemset in df['itemsets']:
        cleaned_string = itemset.replace("[", "").replace("]", "").replace("'", "")
        items = cleaned_string.split(', ')
        items_list = list(map(str.strip, items))  # This removes leading/trailing spaces
        if items_list[0] in grouped_phrases:
            grouped_phrases[items_list[0]].append(items_list[1])
        else:
            grouped_phrases[items_list[0]] = [items_list[1]]
    return grouped_phrases

# Group itemsets by starting word
grouped_phrases = group_by_starting_word(data_group)
print(grouped_phrases)

In [ ]:
import networkx as nx
# Show the 
def plot_graph(key, values):
    G = nx.Graph()

    # Add main node (key)
    G.add_node(key)

    # Add edges between main node and each phrase (value)
    for value in values:
        G.add_edge(key, value)

    # Plotting the graph
    pos = nx.spring_layout(G, seed=42)  # positions for all nodes

    plt.figure(figsize=(8, 6))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=10, font_weight='bold', arrows=False)
    plt.title(f'Undirected Graph for "{key}"')
    plt.show()

# Iterate through each key-value pair and plot the graph
for key, values in grouped_phrases.items():
    plot_graph(key, values)

### Què passa si no normalitzem o comptem els stop words?

In [ ]:
data_set_with_stop_words = generate_data_set(result.sample(frac =.30), rmStopWords = False, useStandard = True)
frequent_itemsets = get_frequent_itemsets(data_set_with_stop_words, 0.001)
# Sort them from more length and then support
frequent_itemsets_sorted = frequent_itemsets.sort_values(by=['length', 'support'], ascending=[False, False])
print(frequent_itemsets_sorted)

In [ ]:
data_group = frequent_itemsets[(frequent_itemsets['length'] >= 2)]
data_group['itemsets'] = data_group['itemsets'].apply(lambda x: str(sorted(list(x))))
data_group['support messages'] = data_group['support'].apply(lambda x: math.ceil(len(partial_data_set) * x))
data_group = data_group.sort_values(by=['support messages'], ascending=False)

print(data_group[['itemsets', 'support messages']])

In [ ]:
grouped_phrases = group_by_starting_word(data_group)
print(grouped_phrases)
# Iterate through each key-value pair and plot the graph
for key, values in grouped_phrases.items():
    plot_graph(key, values)

In [ ]:
data_set_no_standard = generate_data_set(result.sample(frac =.5), rmStopWords = False, useStandard = False)
frequent_itemsets = get_frequent_itemsets(data_set_no_standard, min_supp = 0.005) 

# Sort them from more length and then support
data_group = frequent_itemsets[(frequent_itemsets['length'] >= 2)]
data_group['itemsets'] = data_group['itemsets'].apply(lambda x: str(sorted(list(x))))
data_group['support messages'] = data_group['support'].apply(lambda x: math.ceil(len(partial_data_set) * x))
data_group = data_group.sort_values(by=['support messages'], ascending=False)

print(data_group[['itemsets', 'support messages']])

In [ ]:
grouped_phrases = group_by_starting_word(data_group)
print(grouped_phrases)
# Iterate through each key-value pair and plot the graph
for key, values in grouped_phrases.items():
    plot_graph(key, values)

Per limitacions d'espai, i seguint el consell del professor, hem decidit utilitzar fragments parcials del conjunt de dades en lloc de calcular l'algoritme amb tot el data set.

El resultat s'ha decidit mostrar usant graphs, ja que l'algoritme d'ítems freqüents concentra la major quantitat de grups en els paquets de 2 i és fàcil de veure com una paraula, es relaciona amb una altra usant aquesta.
I el que hem obtingut, es el conjunt de paraules clau que solen apareixer junts. Per exemple, per fix, tenim:

`'fix': ['in', 'to', 'for', 'on', 'the', 'of', 'issue', 'with']`

El qual significa, que les persones solen escriure algo com:
- "fix xxxx to yyyyy"
- "fix ssss with nnnn"
- "fix issue eeee"


### Exploració sintàctica

En aquesta secció, busquem identificar les categories gramaticals més comunes o dominants dels missatges.

In [ ]:
from nltk import word_tokenize, pos_tag
from collections import Counter

# Uncomment to dowload the necessary data to run the functions.
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

In [ ]:
# Map to make the tags more reaedable //https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
tag_map_cat = {
    'CC': 'Conjunció coordinativa',
    'CD': 'Nombre cardinal',
    'DT': 'Determinant',
    'EX': 'Hi ha existencial',
    'FW': 'Paraula estrangera',
    'IN': 'Preposició o conjunció subordinant',
    'JJ': 'Adjectiu',
    'JJR': 'Adjectiu comparatiu',
    'JJS': 'Adjectiu superlatiu',
    'LS': 'Marcador ítems de llista ',
    'MD': 'Modal',
    'NN': 'Nom, singular o massa',
    'NNS': 'Nom, plural',
    'NNP': 'Nom propi, singular',
    'NNPS': 'Nom propi, plural',
    'PDT': 'Predeterminant',
    'POS': 'Sufix possessiu',
    'PRP': 'Pronom personal',
    'PRP$': 'Pronom possessiu',
    'RB': 'Adverbi',
    'RBR': 'Adverbi comparatiu',
    'RBS': 'Adverbi superlatiu',
    'RP': 'Partícula',
    'SYM': 'Símbol',
    'TO': 'a',
    'UH': 'Interjecció',
    'VB': 'Verb, forma base',
    'VBD': 'Verb, pretèrit',
    'VBG': 'Verb, gerundi o participi present',
    'VBN': 'Verb, participi passat',
    'VBP': 'Verb, present no tercera persona singular',
    'VBZ': 'Verb, present tercera persona singular',
    'WDT': 'Determinant WH',
    'WP': 'Pronom WH',
    'WP$': 'Pronom possessiu WH',
    'WRB': 'Adverbi WH'
}
# English version
tag_map_en = {
    'CC': 'Coordinating conjunction',
    'CD': 'Cardinal number',
    'DT': 'Determiner',
    'EX': 'Existential there',
    'FW': 'Foreign word',
    'IN': 'Preposition or subordinating conjunction',
    'JJ': 'Adjective',
    'JJR': 'Adjective, comparative',
    'JJS': 'Adjective, superlative',
    'LS': 'List item marker',
    'MD': 'Modal',
    'NN': 'Noun, singular or mass',
    'NNS': 'Noun, plural',
    'NNP': 'Proper noun, singular',
    'NNPS': 'Proper noun, plural',
    'PDT': 'Predeterminer',
    'POS': 'Possessive ending',
    'PRP': 'Personal pronoun',
    'PRP$': 'Possessive pronoun',
    'RB': 'Adverb',
    'RBR': 'Adverb, comparative',
    'RBS': 'Adverb, superlative',
    'RP': 'Particle',
    'SYM': 'Symbol',
    'TO': 'to',
    'UH': 'Interjection',
    'VB': 'Verb, base form',
    'VBD': 'Verb, past tense',
    'VBG': 'Verb, gerund or present participle',
    'VBN': 'Verb, past participle',
    'VBP': 'Verb, non-3rd person singular present',
    'VBZ': 'Verb, 3rd person singular present',
    'WDT': 'Wh-determiner',
    'WP': 'Wh-pronoun',
    'WP$': 'Possessive wh-pronoun',
    'WRB': 'Wh-adverb'
}

# Load all text
all_text = " ".join(commit for commit in df_commits.message)
print("Totes les paraules:", len(all_text))

In [ ]:
def calc_ratios(all_text, tag_map):
    # apply tokenization to the text and tag it
    tokens = word_tokenize(all_text)
    tagged = pos_tag(tokens)
    
    # Count the occurrence of each tag
    tag_counts = Counter(tag for word, tag in tagged)
    
    # Convert tags to full wording in Catalan
    english_tag_counts = {tag_map.get(tag, tag): count for tag, count in tag_counts.items()}
    
    # Calculate the ratio of each tag
    total_tags = sum(tag_counts.values())
    tag_ratios = {tag: count / total_tags for tag, count in english_tag_counts.items()}
    
    # Sort ratios by values in descending order
    sorted_ratios = sorted(tag_ratios.items(), key=lambda item: item[1], reverse=True)

    # Create a dataframe to show the result
    df_ratios = pd.DataFrame(sorted_ratios, columns=['Etiqueta', 'Ratio'])
    
    # Show the equivalence of ratio in words
    data_size = len(all_text)
    df_ratios['Paraules'] = df_ratios['Ratio'].apply(lambda x: math.ceil(x * data_size))
    df_ratios['Ratio'] = df_ratios['Ratio'].apply(lambda x: (x * 100)) # Put it in percentage
    
    return df_ratios

In [ ]:
titles_cat = ['Categoria gramatical o símbols', 'Percentatge (%)', 'Distribució ús de categories gramaticals']
titles_en = ['Grammatical category or symbols', 'Percentage (%)', 'Distribution of use of grammatical categories']

titles = titles_cat
tag_map = tag_map_cat
    
df_ratios = calc_ratios(all_text, tag_map)
print(df_ratios)

In [ ]:
# Plotting the ratios
tags = df_ratios["Etiqueta"]
ratios = df_ratios["Ratio"]

plt.figure(figsize=(10, 6))
plt.bar(tags, ratios, color='skyblue')
plt.xlabel(titles[0])
plt.ylabel(titles[1])
plt.title(titles[2])
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

A través dels resultats, podem destacar que hi ha una dominància dels noms: Les categories "Nom, singular o massa" i "Nom propi, singular" ocupen les dues primeres posicions en termes de freqüència relativa (ratio). Això podria indicar que solen descriure o senyalar molt a entitas concrets i per això potser tenim en tercera posició els adejctius. 

Per tant, es podria dir que els missatges són més aviat descriptives, amb l'objectiu de facilitar i saber a què fan referència. 

### I quines formes verbals s'usen més?

In [ ]:
# Calculate ratio of verbs
data_size = len(all_text)
verb_usage = df_ratios[df_ratios['Etiqueta'].str.startswith('Verb')].copy()
verb_usage['Paraules'] = verb_usage['Ratio'].apply(lambda x: math.ceil(x * len(all_text)))
print(verb_usage)

In [ ]:
titles_cat = ['Temps verbals dels verbs', 'Percentatge (%)', 'Distribució ús de categories gramaticals']
titles_en = ['Verb tenses of verb', 'Percentage (%)', 'Verb usage distribution']

titles = titles_en

In [ ]:
tags = verb_usage['Etiqueta']
ratios = verb_usage['Ratio']

print("Total percentage of verbs: {:.2f}%".format(verb_usage['Ratio'].sum()))
print("Total verbs:", verb_usage['Paraules'].sum())

# Plotting the ratios
plt.figure(figsize=(10, 6))
plt.bar(tags, ratios, color='skyblue')
plt.xlabel(titles[0])
plt.ylabel(titles[1])
plt.title(titles[2])
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

Per tant, com a resultat, podem concloure que els missatges estan enfocats a clarificar i identificar els canvis realitzats en el codi. Ja que els verbs, ocupen un ~8% i l'altre part, queda dominada sobretot per noms, adjectius i simbols que els quals tots tene un enfoc de clarificar i assenyalar. Aquest enfocament és crucial per a la comunicació eficient entre els membres de l'equip de desenvolupament i per a la comprensió precisa de les modificacions implementades.

# I si ho fem per missatges?
Ara veurem, la mitjana, la desviació i la mediana d'un missatge en termes de paraules usades. I farem el comput de la mitja de percentatge de ús gramatical segons missatges.

In [ ]:
average_words = df_commits['message'].apply(lambda x: len(x.split())).mean()
std_dev_words = df_commits['message'].apply(lambda x: len(x.split())).std()
median_words = df_commits['message'].apply(lambda x: len(x.split())).median()

print(f"The average message of a commit has {math.ceil(average_words)} words. With a standard deviation of {math.ceil(std_dev_words)} words and a median of {median_words}.")

In [ ]:
def calc_ratios_message(df_commits, tag_map, average_words):
    all_ratios = []

    for message in df_commits.message:
        # apply tokenization to the text and tag it
        tokens = word_tokenize(message)
        tagged = pos_tag(tokens)

        # Count the occurrence of each tag
        tag_counts = Counter(tag for word, tag in tagged)

        # Convert tags to full wording in Catalan
        english_tag_counts = {tag_map.get(tag, tag): count for tag, count in tag_counts.items()}

        # Calculate the ratio of each tag
        total_tags = sum(tag_counts.values())
        tag_ratios = {tag: count / total_tags for tag, count in english_tag_counts.items()}

        all_ratios.append(tag_ratios)

    # Calculate the average ratios
    avg_ratios = pd.DataFrame(all_ratios).mean().sort_values(ascending=False)

    # Create a dataframe to show the result
    df_ratios = pd.DataFrame(avg_ratios.reset_index())
    df_ratios.columns = ['Etiqueta', 'Ratio']

    # Show the equivalence of ratio in words
    data_size = len(df_commits)
    df_ratios['Paraules'] = df_ratios['Ratio'].apply(lambda x: math.ceil(x * average_words))
    df_ratios['Ratio'] = df_ratios['Ratio'].apply(lambda x: (x * 100)) # Put it in percentage

    return df_ratios

In [ ]:
titles_cat = ['Categoria gramatical o símbols', 'Percentatge (%)', 'Categoria gramatical mitjana d\'un missatge en termes de paraules utilitzades']
titles_en = ['Grammatical category or Symbols', 'Percentage (%)', 'Average grammatical category of a message in terms of words used']

titles = titles_cat
tag_map = tag_map_cat

df_average_ratio_per_message = calc_ratios_message(df_commits, tag_map, average_words)
print(df_average_ratio_per_message)

In [ ]:
# Plotting the ratios
tags = df_average_ratio_per_message["Etiqueta"]
ratios = df_average_ratio_per_message["Ratio"]

plt.figure(figsize=(10, 6))
plt.bar(tags, ratios, color='skyblue')
plt.xlabel(titles[0])
plt.ylabel(titles[1])
plt.title(titles[2])
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
titles_cat = ['Temps verbals dels verbs', 'Percentatge (%)', 'Distribució ús de verbs en un missatge segons temps verbal']
titles_en = ['Verb tenses of verb', 'Percentage (%)', 'Verb usage distribution in a message based on verb tense']

titles = titles_en

In [ ]:
# Calculate ratio of verbs
verb_usage = df_average_ratio_per_message[df_average_ratio_per_message['Etiqueta'].str.startswith('Verb')].copy()
print(verb_usage)

tags = verb_usage['Etiqueta']
ratios = verb_usage['Ratio']

# Plotting the ratios
plt.figure(figsize=(10, 6))
plt.bar(tags, ratios, color='skyblue')
plt.xlabel(titles[0])
plt.ylabel(titles[1])
plt.title(titles[2])
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

Podem veure que en un missatge, els noms, segueixen dominant la distribució de la categoria gramatical en un commit. Indicant, per exemple, que si tenim un missatge de 6 paraules, el ~37% de les paraules, són noms, singular o massa. A diferència de la distribució de totes les paraules, podem destacar, que els verbs, ara tenent més protagonisme, on els pretèrits i participi passat solen apareixer més. Això suggerix que els missatges de commit sovint descriuen accions que s'han completat.

# Altres analisis no-conclusius
Per deficiències en la recollida de dades desafortunadament hi ha altres anàlisis que no hem pogut fer. A continuació mostrem com es farien si tinguéssim les dades necessàries.

El primer d'aquests era visualitzar l'evolució de la quantitat de repositoris en el temps per cada topic. Desafortunadament hem perdut la majoria de les dades recollides; només les tenim per uns pocs dies, i per tant la gràfica no es gaire útil.

In [ ]:
_, df_topic_visits = execute_select_query(cursor, "SELECT CAST(date AS Date) AS date, name, repositories, followers FROM TopicVisits;")
df_topic_visits_n_cut = df_topic_visits[df_topic_visits["name"].isin(n_topics_name)]
df_topicv_repos = df_topic_visits_n_cut[["date", "name", "repositories"]]
n_topics_name
display(df_topic_visits_n_cut)

In [ ]:
df_topicv_repos.pivot_table(index='date', columns="name", values="repositories", aggfunc="sum").plot()

Volem ara veure una evolució de la popularitat dels repositoris tractant els temes que hem identificat com a principals. És a dir, volem veure com ha anat canviant el nombre d'estrelles que reben aquests repositoris. Aquesta és una bona oportunitat que tenim per a treballar els joins entre taules.

In [ ]:
query = """
WITH RepositoriesMainTopics AS (
	SELECT repo AS name, topic
    FROM RepositoryTopics
    WHERE topic IN ('hacktoberfest', 'javascript', 'react', 'typescript', 'nodejs', 'python', 'docker', 'nextjs', 'golang', 'go', 'git', 'minecraft', 'java', 'vue', 'machine-learning', 'android', 'deep-learning', 'github', 'ios', 'cli')
), TrendingTopic AS (
	SELECT date, repoName AS name, topic, starsToday as stars
    FROM RepositoriesMainTopics
    JOIN TrendVisits ON TrendVisits.repoName = RepositoriesMainTopics.name
)
SELECT CAST(date AS Date) AS date, topic, SUM(stars) AS stars
FROM TrendingTopic
GROUP BY date, topic
"""
_, df_trendRepositoriesMainTopics = execute_select_query(cursor, query)
df_trendRepositoriesMainTopics.head(5)

In [ ]:
df_trendRepositoriesMainTopics["stars"] = df_trendRepositoriesMainTopics["stars"].astype(float)
df_trendRepositoriesMainTopics.dtypes

In [ ]:
df_trendRepositoriesMainTopics.pivot_table(index='date', columns="topic", values="stars", aggfunc="sum").plot()

In [ ]:
# Closes the connection
dataBaseConnection.close()